In [56]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2

In [62]:
train_dataset = image_dataset_from_directory("../data/Train/",
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(224, 224),
                                             validation_split=0.2,
                                             subset='training',
                                             seed=1)
validation_dataset = image_dataset_from_directory("../data/Train/",
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(224, 224),
                                                  validation_split=0.2,
                                                  subset='validation',
                                                  seed=1)

Found 3679 files belonging to 10 classes.
Using 2944 files for training.
Found 3679 files belonging to 10 classes.
Using 735 files for validation.


In [63]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

def skinModel():
    # Use MobileNetV3Large with the correct input size
    baseModel = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    baseModel.trainable = False  # Freeze the base model's layers

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = preprocess_input(inputs)  # Preprocess input for MobileNetV3
    x = baseModel(x, training=False)  # Pass inputs through MobileNetV3
    x = GlobalAveragePooling2D()(x)  # Pool the feature maps
    x = BatchNormalization()(x)  # Apply Batch Normalization
    x = Dropout(0.3)(x)  # Apply Dropout for regularization
    outputs = Dense(10, activation='softmax')(x)  # Final classification layer with 10 classes

    return Model(inputs, outputs)

# Create the MobileNetV3Large-based model
model = skinModel()

In [68]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=20, validation_data=validation_dataset)

Epoch 1/20
92/92 [==============================] - 82s 845ms/step - loss: 0.5481 - accuracy: 0.7999 - val_loss: 0.9470 - val_accuracy: 0.7197
Epoch 2/20
92/92 [==============================] - 85s 921ms/step - loss: 0.5587 - accuracy: 0.8057 - val_loss: 0.9456 - val_accuracy: 0.7061
Epoch 3/20
36/92 [==========>...................] - ETA: 1:01 - loss: 0.5378 - accuracy: 0.7977

KeyboardInterrupt: 

In [71]:
model.save('skinModel2.h5', save_format='h5')


In [70]:
test_dataset = image_dataset_from_directory("../data/Test/",
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(224, 224))

model.evaluate(test_dataset)

Found 478 files belonging to 10 classes.
15/15 [==============================] - 13s 696ms/step - loss: 0.7716 - accuracy: 0.8096


[0.7716163992881775, 0.8096234202384949]

In [67]:
from tensorflow.keras.models import load_model

model1=load_model('skinModel2.h5')